### Customer Dimension

In [0]:
%sql

Create or replace table datamodeling.gold.DimCustomer as  
(
with cte1 as(
    Select 
    distinct customer_id,customer_email,customer_name, customer_namr_upper
    from datamodeling.silver.silvertable
  )
Select *, row_number() over (order by customer_id) as DimCustomerKey 
from cte1
)

In [0]:
%sql
SELECT * FROM datamodeling.gold.DimCustomer

### Dim Product

In [0]:
%sql
Create or replace table datamodeling.gold.DimProduct as
(
   with cte1 as(
   Select distinct Product_id, Product_name, product_category 
   from datamodeling.silver.silvertable
   ) 
Select *, row_number() over (order by product_id) as DimProductkey 
  from cte1
)

In [0]:
%sql
Select * from datamodeling.gold.DimProduct

### Dim Payment

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimPayment as 
(
With cte1 as(
 Select distinct payment_type from datamodeling.silver.silvertable
) Select *, row_number() over (order by payment_type) as DimPaymentKey from cte1
)

In [0]:
%sql
Select * from datamodeling.gold.DimPayment

### Dim Region

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimRegion as 
(
With cte1 as(
 Select distinct Country from datamodeling.silver.silvertable
) Select *, row_number() over (order by country) as DimRegionKey from cte1
)

In [0]:
%sql
Select * from datamodeling.gold.DimRegion

### Table that has theme of Whole table 
### Dim Sales

In [0]:
spark.sql("Select * from DataModeling.default.sourcedata").columns 

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimSales as
(
Select 
row_number() over (order by order_id) as DimSalesKey,
order_id,
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated
 from DataModeling.default.sourcedata
)

In [0]:
%sql
Select * from datamodeling.gold.DimSales

### Fact Table - numeric column + keys

In [0]:
%sql
Create or replace table datamodeling.gold.fact_sales as
(
Select 
 quantity, unit_price,
 DimSalesKey
 DimCustomerKey,
 DimPaymentKey,
 DimRegionKey,
 DimProductKey
 from datamodeling.silver.silvertable as st
 left join
  datamodeling.gold.DimCustomer as DimCust on
 st.Customer_id = DimCust.customer_id
 left join 
datamodeling.gold.DimPayment as DimPay on
 st.payment_type = DimPay.payment_type
left join 
datamodeling.gold.DimRegion as DimReg on
 st.country = DimReg.country
left join 
datamodeling.gold.DimProduct as DimProd on
 st.product_id = DimProd.product_id
 left join 
datamodeling.gold.DimSales as Dimsales on
 st.order_id = Dimsales.order_id
)

 

In [0]:
%sql
Select * from datamodeling.gold.fact_sales